In [ ]:
%%bash
python3 -m pip install kafka-python rx jupyternotify

In [ ]:
%load_ext jupyternotify

In [ ]:
import json
from kafka import KafkaConsumer, KafkaProducer
from kafka.common import KafkaError
from rx import Observable


KAFKA_BROKERS = '10.10.139.63:9092'
KAFKA_TOPICS = ['calculated.ddos2222', 'calculated.forbidden2']
KAFKA_OFFSET_RESET = 'earliest' # TODO


kafka_consumer = KafkaConsumer(auto_offset_reset=KAFKA_OFFSET_RESET,
                               bootstrap_servers=KAFKA_BROKERS,
                               enable_auto_commit=False, # TODO
                               # group_id=KAFKA_GROUP,
                               value_deserializer=lambda value: json.loads(value.decode('utf-8')))
kafka_consumer.subscribe(KAFKA_TOPICS)

# We receive flagged counts one by one from spark so we have to keep a data structure to match true and false flagged.
# Structure:
# {
#     'timestamp': {
#         'flagged_true': 7,
#         'flagged_false': 1,
#     }
# }
window_cache = {}


def extract_window_key(message):
    return '{}-{}'.format(message['window']['start'], message['window']['start'])


def add_to_window_cache(message):
    '''
    message structure:
    {"window":{"start":"2018-01-13T05:04:00.000Z","end":"2018-01-13T05:04:10.000Z"},"flagged":true,"count":7}
    '''
    key = extract_window_key(message)
    if key not in window_cache:
        window_cache[key] = {}
        
    if message['flagged']:
        window_cache[key]['flagged_true'] = message['count']
    else:
        window_cache[key]['flagged_false'] = message['count']
    return len(window_cache[key]) == 2
    

def ddos_window_detection(message):
    key = extract_window_key(message)
    # For this exercise we will keep the "detection" very simple by calculting the ratio of (true / total) > threshold
    ddos_ratio = window_cache[key]['flagged_true'] / (window_cache[key]['flagged_false'] + window_cache[key]['flagged_true'])
    if ddos_ratio > 0.95:
        # Trigger alert, possible ddos
        return True
    # If no suspicious activity was found, we can delete the window from the cache
    del window_cache[key]
    return False
    

def notify_admin_ddos(message):
    key = extract_window_key(message)
    window = window_cache.pop(key)
    %notify -m 'DDOS alert'
    
def notify_admin_forbidden_url(message):
    %notify -m 'Forbidden URL alert'
    
    
# {"window":{"start":"2018-01-13T05:04:00.000Z","end":"2018-01-13T05:04:10.000Z"},"flagged":true,"count":7}
for msg in kafka_consumer:
    if msg.topic == 'calculated.ddos2222':
        if add_to_window_cache(msg.value) and ddos_window_detection(msg.value):
            notify_admin_ddos(msg.value)
    elif msg.topic == 'calculated.forbidden2':
        if msg.value['forbidden']:
            notify_admin_forbidden_url(msg.value)

